# Colouring-in practice with Pandas Dataframes
Trying out some ways of styling dataframes

**Note the GitHub ipynb renderer may be unable to display the styling below!**

### Create pandas dataframes

In [1]:
import pandas as pd

data = {
'one':  [1., 2.,  3.,   4., "n"],
'two':  [2., 4., -6.,   8., "a"],
'three':[3., 6., -9., -12., "one"]    
}

#Create a dataframe and display it
df = pd.DataFrame(data)
display(df)

,one,three,two
0,1,3,2
1,2,6,4
2,3,-9,-6
3,4,-12,8
4,n,one,a


In [2]:
#Another dataframe - this one has generated values
data = {"x": range(0,11)}
df2 = pd.DataFrame(data)
df2["y"] = df2["x"]**2
df2["z"] = 5*df2["x"] - df2["y"]
display(df2)

,x,y,z
0,0,0,0
1,1,1,4
2,2,4,6
3,3,9,6
4,4,16,4
5,5,25,0
6,6,36,-6
7,7,49,-14
8,8,64,-24
9,9,81,-36


## General table style setting

Can use style.set_table_styles to set general values

- Does not pay attention to data within the table    
- argument is list of dictionaries    
- each dictionary has a "selector" key - with css element (str) and "props" key with list containing element,value tuple pairs 
- returns a Styler object which can be displayed like a dataframe but lack many of a dataframe's features and cannot itself be further styled

In [3]:
styles = [
    {"selector":"th", "props":[("font-size","120%"),("color","cyan"),("background-color","grey")]}
]

new = df.style.set_table_styles(styles)
display(new)


,one,three,two
0,1,3,2
1,2,6,4
2,3,-9,-6
3,4,-12,8
4,n,one,a


#### **Alternative approach** - style.set_properties
Can also use style.set_properties to set some general values

In [4]:
newborders = df.style.set_properties(**{'border-color': 'black', 'border-style':'solid', 'border-width':'thin'})
display(newborders)

,one,three,two
0,1,3,2
1,2,6,4
2,3,-9,-6
3,4,-12,8
4,n,one,a


## Conditional Highlighting

More than one way to do this. Can apply highlighing based on individual cell's own content alone or in a row-wise or column-wise approach.

### Cell-based conditional colour highlighting
df.style.applymap can be used to apply highlighting based on a cell's contents.     
Requires a function that defines the highlight conditions.
Two different highlighter functions below.

In [5]:
def highlight_conditions(val):
    """
    Colour selection function intended to be used with
    df.style.applymap to apply conditional colours to 
    dataframe values.
    Based on info from 
    https://pandas.pydata.org/pandas-docs/stable/style.html
    """
    # Default text colour
    colour = 'black'
    
    # Conditional colours
    if val<0: colour = 'red'
    elif val == "z": colour = 'orange'
    elif val == "one": colour = 'green'
    
    # Create the and return the 
    # css element that df.style.applymap uses
    return 'color: %s' % colour

def highlight(val):
    """Different way of mapping the colours"""
    mapping = {
        "CIS-1910":("red", "pink"),
        "CIS-9001":("yellow", "orange"),
        "CIS-2307":("purple", "blue")
    }
    colours = mapping.get(val, ("black", "white") )
    return 'color: {}; background-color: {}'.format(*colours)

def fg_bg_highlight_conditions(val):
    """
    Colour selection function intended to be used with
    df.style.applymap to apply conditional colours to Pandas dataframe values.
    This one can set text colour and background colour
    Based on info from https://pandas.pydata.org/pandas-docs/stable/style.html
    colour names- https://python-graph-gallery.com/100-calling-a-color-with-seaborn/
    """
    #T ext colour mapping
    colour_map = {"FAILED":"red", "RUNNING":"purple"}
    #Retrieve mapped colour (default to black)
    colour = colour_map.get(val, "black")
    
    # Background colour mapping
    bg_map = {"FAILED":"pink", "RUNNING":"yellow"}
    # Retrieve mapped colour (default to black)
    bg = bg_map.get(val, "white")
    
    # Create the and return the css elements that df.style.applymap uses
    return 'color: {}; background-color: {}'.format(colour, bg)

### Create highlighted version and display it

In [6]:
highlighted = df.style.applymap(highlight_conditions)
display(highlighted)

,one,three,two
0,1,3,2
1,2,6,4
2,3,-9,-6
3,4,-12,8
4,n,one,a


### Row-based conditional stying
- Can use df.style.apply method to apply row-based or column-based highlighting (depends on axis parameter)
- When row-based, requires a function that creates conditional format for each column within a row

In [7]:
def row_style(row_data):
    """Row-by-row styler for pandas
    Use with df.style.apply(row_data, axis=1)
    """
    highlighted = "color: blue; background-color: orange"
    
    # Define style for each column within the row
    # List comprehension might be possible but trouble too
    row_styles = []
    for i, value in enumerate(row_data):
        #Default plain style
        style = ""
        #Special styling
        if i==2:
            if row_data[0] < row_data[2]:
                style = highlighted
        row_styles.append(style)
        
    return row_styles

In [8]:
df_style = df2.style.apply(row_style, axis=1)
display(df_style)

,x,y,z
0,0,0,0
1,1,1,4
2,2,4,6
3,3,9,6
4,4,16,4
5,5,25,0
6,6,36,-6
7,7,49,-14
8,8,64,-24
9,9,81,-36


### Applying multiple stylings
- Using set_table_styles, applymap, apply, and set_properties style methods.     
- All have to be done at once because the returned Styler object cannot itself be further styled.

In [9]:
both = df.style.set_table_styles(styles).applymap(highlight_conditions).apply(row_style, axis=1).set_properties(**{'border-color': 'black', 'border-style':'solid', 'border-width':'thin'})
display(both)

,one,three,two
0,1,3,2
1,2,6,4
2,3,-9,-6
3,4,-12,8
4,n,one,a


### Styler to HTML
add .render() when applying styles to dataframe

In [12]:
html_table = df.style.set_table_styles(styles)\
.applymap(highlight_conditions).apply(row_style, axis=1)\
.set_properties(**{'border-color': 'black', 'border-style':'solid', 'border-width':'thin'})\
.render()

In [14]:
from IPython.core.display import  HTML
display(HTML(html_table))

,one,three,two
0,1,3,2
1,2,6,4
2,3,-9,-6
3,4,-12,8
4,n,one,a


In [15]:
html_table

u'<style  type="text/css" >\n    #T_f9d74dac_b981_11e8_9097_f81654995aa7 th {\n          font-size: 120%;\n          color: cyan;\n          background-color: grey;\n    }    #T_f9d74dac_b981_11e8_9097_f81654995aa7row0_col0 {\n            color:  black;\n            : ;\n            border-color:  black;\n            border-style:  solid;\n            border-width:  thin;\n        }    #T_f9d74dac_b981_11e8_9097_f81654995aa7row0_col1 {\n            color:  black;\n            : ;\n            border-color:  black;\n            border-style:  solid;\n            border-width:  thin;\n        }    #T_f9d74dac_b981_11e8_9097_f81654995aa7row0_col2 {\n            color:  black;\n            color:  blue;\n             background-color:  orange;\n            border-color:  black;\n            border-style:  solid;\n            border-width:  thin;\n        }    #T_f9d74dac_b981_11e8_9097_f81654995aa7row1_col0 {\n            color:  black;\n            : ;\n            border-color:  black;\n

### Might add something about comparing two dataframes

In [ ]:
for column in df:
    print column

In [ ]:
Hmm
https://stackoverflow.com/questions/29464234/compare-python-pandas-dataframes-for-matching-rows

In [16]:
username = raw_input("Username: ")

Username: cake


In [1]:
import getpass
password = getpass.getpass("Password:")